In [ ]:
# The notebook evaluates the meeting summary model on samsum dataset using the rouge model
from tqdm import tqdm
import torch
from datasets import load_metric, load_dataset
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

rouge_metric = load_metric("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]
        
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size = 16, device = device, column_text = "article", column_summary = "highlights", max_summmary_length = 128):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))
    
    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total = len(article_batches)):
        inputs = tokenizer(article_batch, max_length = 1024, truncation = True, padding = "max_length", return_tensors = "pt")
        summaries = model.generate(input_ids = inputs["input_ids"].to(device), attention_mask = inputs["attention_mask"].to(device), length_penalty = 0.8, num_beams = 8, max_length = max_summmary_length)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions = decoded_summaries, references = target_batch)
    score = metric.compute()
    return score

In [ ]:
# Load the model and dataset from hub
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_ckpt = "tanviraumi/meeting-summary"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
dataset_samsum = load_dataset("samsum")

In [5]:
# Run the evaluation on pegasus test data. First we use the default summary length of 128
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model, tokenizer, batch_size = 2, column_text = "dialogue", column_summary = "summary", max_summmary_length = 128)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = ["pegasus-test-data"])

100%|█████████████████████████████████████████████████████████████████████████████████████| 410/410 [04:12<00:00,  1.63it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus-test-data,0.48389,0.254176,0.412378,0.412114


In [6]:
# Run the evaluation on pegasus test data. This time we use a length of 256
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model, tokenizer, batch_size = 2, column_text = "dialogue", column_summary = "summary", max_summmary_length = 256)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = ["pegasus-test-data"])

100%|█████████████████████████████████████████████████████████████████████████████████████| 410/410 [04:15<00:00,  1.61it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus-test-data,0.48389,0.254176,0.412378,0.412114


In [10]:
# Now lets try some custom conversations
from transformers import pipeline
pipe = pipeline("summarization", model = model_ckpt)
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 256}

custom_conversation = """\
Justen: No worries. It's all good. Hey, Kelli. How's it going? Hey, good.\nKelli: How are you? Good? How was your extended weekend?\nJusten: I didn't hear what you said, sorry.\nKelli: Sorry, that thing is obnoxious.\nJusten: How was your extended weekend? It was good.\nKelli: I did nothing, which is what I had planned to do that's great. How are you? Yeah, we both are Highspot gave us the Friday off, which was really nice.\nJusten: We got the four day weekend and it has really relaxing. How about you? Isreal… I have two daughters, one going into six grade one going into fourth.\nIsreal: So we had some gym shoes purchasing. We had some net grandparents. Yeah, we're in kind of a high. Situation.\nApryl: Where they'll go to school half the time and at home behind the iPad half the time.\nIsreal: So… I definitely respect the virus, but I also think kids that it's probably need some socialization and they're gonna turn out quote unquote normal, probably need to be in class and learn how to be a good human. So… it's exciting to see him go back, but also very nerve racking and just trying to juggle work schedules with at home schedules as stressful. But yeah, we use the time before they started yesterday. So we just follow the grandparents and got all ran all the errands we needed to… to be ready for another successful school year. Well… Matt… Matt prompts to all the parents out there because it's hard enough for me to… to work from home by myself.\nJusten: So I can only imagine that if you have kids running around and… and also. Just took all nightmare, I think for the schools because plans were even being changed like Sunday and Monday before, at lunch Tuesday.\nIsreal: So a lot of moving parts for sure. Yeah.\nApryl: I know here in Seattle, I have a couple of friends that are teachers. And if they teach in Seattle metro is not even just the… the uncertainty of how they're going to structure it, but typically in Seattle, you don't know if you're going to be rehire for a contract as a teacher until about five days before school starts. So they are waiting on a couple of different, the way, am I gonna have a job? And then after that, how am I going to do my job? So it's a little stressful. Yeah.\nIsreal: I… I love the hybrid piece. My kids one's very academic and studious, the others does… does well, but it's probably more middle of the class and does just more into the social aspects. And Ruby, she struggled like I thought you it, but I'll always is like top 98 percent when she does the standardized testing and even she got sick of learning behind the iPad."""
print(pipe(custom_conversation, **gen_kwargs)[0]["summary_text"])

Justen, Kelli, Apryl and Isreal had an extended weekend.
